In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
set_config(transform_output='pandas')
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate,StratifiedKFold, cross_val_score, train_test_split
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder,MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

In [66]:
os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
data = pd.read_csv('ML_Data_Teams.csv', sep=';')	

In [67]:
data.columns

Index(['Standard_Namen', '@NoPlayer1', '@NoPlayer2', '@PositionInEntry',
       '@Rank', '@EarnedPointsTeam', '@EarningsTotalTeam', 'Gender_x', 'Type',
       'TournamentNo', 'SpikeFault', 'SpikePoint', 'ServeFault', 'ServePoint',
       'ServeTotal', 'BlockPoint', 'BlockTotal', 'DigTotal', 'ReceptionFault',
       'SpikeTotal', '@LocalDate', '@LocalTime', 'FederationCode_y',
       'FirstName', 'LastName', 'FirstName2', 'LastName2', '@TeamAName',
       '@TeamBName', 'NoPlayer1_team', 'NoPlayer2_team', '@PointsTeamASet1',
       '@PointsTeamBSet1', '@PointsTeamASet2', '@PointsTeamBSet2',
       '@PointsTeamASet3', '@PointsTeamBSet3', '@DurationSet1',
       '@DurationSet2', '@DurationSet3', 'temperature_2m', 'precipitation',
       'wind_speed_10m', 'rain', 'wind_gusts_10m', 'TeamFault_team',
       'match_win', 'TeamDesignation', 'Team1', 'Team2',
       'Standard_Namen_team2'],
      dtype='object')

In [68]:
data1 = data[['@PositionInEntry','SpikePoint',
       '@Rank', '@EarnedPointsTeam', '@EarningsTotalTeam', 'Gender_x', 'Type', 
       'FirstName', 'LastName', 'FirstName2', 'LastName2',   '@DurationSet1',
       '@DurationSet2', '@DurationSet3', 
       'match_win', 'Standard_Namen' , 'Standard_Namen_team2']]
#'TournamentNo', 'SpikeFault',  'ServeFault', 'ServePoint',
       # 'ServeTotal', 'BlockPoint', 'BlockTotal', 'DigTotal', 'ReceptionFault',
       # 'SpikeTotal', '@PointsTeamASet1',
       # '@PointsTeamBSet1', '@PointsTeamASet2', '@PointsTeamBSet2','temperature_2m', 'precipitation',
       # 'wind_speed_10m', 'rain', 'wind_gusts_10m', 
       # '@PointsTeamASet3', '@PointsTeamBSet3', '@LocalDate', '@LocalTime',
       #'TeamFault_team','FederationCode_y',

In [69]:
special_feature = '@DurationSet3'
data2 = data1.copy()
# Erzeuge den Indikator: 1, wenn in @DurationSet3 ein Wert vorhanden ist, sonst 0.
data2[special_feature + '_indicator'] = data2[special_feature].notnull().astype(int)

#Daten für app
data2.to_csv('ML_SpikePoint_OHNEWetter.csv', index=False, sep=';')

In [70]:
data2.columns

Index(['@PositionInEntry', 'SpikePoint', '@Rank', '@EarnedPointsTeam',
       '@EarningsTotalTeam', 'Gender_x', 'Type', 'FirstName', 'LastName',
       'FirstName2', 'LastName2', '@DurationSet1', '@DurationSet2',
       '@DurationSet3', 'match_win', 'Standard_Namen', 'Standard_Namen_team2',
       '@DurationSet3_indicator'],
      dtype='object')

In [71]:
y = data2.pop('SpikePoint')
X = data2.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [72]:
X_train.columns

Index(['@PositionInEntry', '@Rank', '@EarnedPointsTeam', '@EarningsTotalTeam',
       'Gender_x', 'Type', 'FirstName', 'LastName', 'FirstName2', 'LastName2',
       '@DurationSet1', '@DurationSet2', '@DurationSet3', 'match_win',
       'Standard_Namen', 'Standard_Namen_team2', '@DurationSet3_indicator'],
      dtype='object')

In [ ]:
# special_feature = '@DurationSet3'

# # Kopiere die Trainingsdaten, um das Original nicht zu überschreiben
# X_train = X_train.copy()

# # Erzeuge den Indikator: 1, wenn in @DurationSet3 ein Wert vorhanden ist, sonst 0.
# X_train[special_feature + '_indicator'] = X_train[special_feature].notnull().astype(int)

# # Für die Testdaten ebenfalls:
# X_test = X_test.copy()
# X_test[special_feature + '_indicator'] = X_test[special_feature].notnull().astype(int)


In [7]:
# # select categorical and numerical column names
# nominal_features = X.select_dtypes(include=['object']).columns.tolist()
# # Define feature types
# numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# # Create preprocessors for different feature types
# numeric_transformer = Pipeline(steps=[
#     ('imputer', KNNImputer(n_neighbors=5)),
#     ('scaler', StandardScaler())
# ])

# nominal_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# # Build the column transformer
# transformers = [
#     ('num', numeric_transformer, numeric_features),
#     ('nom', nominal_transformer, nominal_features)
# ]

# preprocessor = ColumnTransformer(transformers=transformers)

In [73]:
nominal_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Erstelle eine Liste der allgemeinen numerischen Features, exklusive des speziellen Features und dessen Indikator
general_numeric_features = [col for col in numeric_features if col not in [special_feature, special_feature + '_indicator']]

# Erstelle separate Pipelines:
# 1. Für allgemeine numerische Features mit KNNImputer
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# 2. Für das spezielle Feature, das immer mit 0 imputiert werden soll (das signalisiert, dass kein dritter Satz stattgefunden hat)
special_numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# 3. Für die Indikatorspalte (@DurationSet3_indicator)
indicator_transformer = Pipeline(steps=[
    # Hier ist meist keine Imputation nötig; Skalierung ist optional, da 0 und 1 oft schon aussagekräftig sind.
    ('scaler', StandardScaler())
])

# 4. Für nominale Features: Imputation mit dem häufigsten Wert + OneHotEncoding
nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Baue den ColumnTransformer unter Berücksichtigung aller Feature-Gruppen:
transformers = [
    ('num', numeric_transformer, general_numeric_features),
    ('spec_num', special_numeric_transformer, [special_feature]),
    ('indicator', indicator_transformer, [special_feature + '_indicator']),
    ('nom', nominal_transformer, nominal_features)
]

preprocessor = ColumnTransformer(transformers=transformers)

In [74]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

model_scores = []
def score_model(model_name, y_true, y_pred):
  scores = {
      'Model': model_name,
      'MAE ($)': round(mean_absolute_error(y_true, y_pred), 2),
      'RMSE ($)': round(root_mean_squared_error(y_true, y_pred), 2),
      'MAPE (%)': round(100 * mean_absolute_percentage_error(y_true, y_pred), 2),
      'R-Squared': round(r2_score(y_true, y_pred), 3)
  }
  return scores

In [75]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             GradientBoostingRegressor(random_state=42, n_estimators=200, learning_rate=0.1, 
                                                  max_depth=5, subsample=0.8))

GB = gbr_pipeline.fit(X_train, y_train)

gbr_predictions = gbr_pipeline.predict(X_test)

model_scores.append(score_model('GradientBoostigRegressor', y_test, gbr_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,3.08,4.02,1.391732e+16,0.678


In [76]:
train_r2 = gbr_pipeline.score(X_train, y_train)
test_r2  = gbr_pipeline.score(X_test, y_test)
print("Train R²:", train_r2)
print("Test R²:", test_r2)

Train R²: 0.7841118995394285
Test R²: 0.6781698967292412


In [77]:
import joblib
# Modell speichern als .pkl-Datei
joblib.dump(GB, "GradientBoosting_SpikePoint_OHNEWetter_OhneStatistikDaten.pkl")

['GradientBoosting_SpikePoint_OHNEWetter_OhneStatistikDaten.pkl']

In [78]:

# 1. Extrahiere das trainierte Model (der letzte Schritt in der Pipeline)
model = gbr_pipeline.named_steps['gradientboostingregressor']

# 2. Extrahiere den Preprocessor-Schritt, der die Features formatiert hat.
# Je nachdem, wie deine Pipeline aufgebaut ist, kann der Schritt-Name variieren.
# Beim make_pipeline werden die Namen automatisch generiert:
preprocessor = gbr_pipeline.named_steps[list(gbr_pipeline.named_steps.keys())[1]]
# Alternativ, wenn du die Pipeline explizit mit Namen erstellt hast, z.B. im Pipeline-Konstruktur,
# dann wäre es etwas wie:
preprocessor = gbr_pipeline.named_steps['columntransformer']

# 3. Hole die Feature-Namen aus dem Preprocessor:
# Diese Methode funktioniert, wenn der Preprocessor und seine Zwischenschritte get_feature_names_out unterstützen.
feature_names = preprocessor.get_feature_names_out()

# 4. Extrahiere die Feature Importances aus dem Model
importances = model.feature_importances_

# 5. Erstelle einen DataFrame zum besseren Überblick
feature_importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importances_df.head(10))


                               feature  importance
9              spec_num__@DurationSet3    0.519627
8                       num__match_win    0.087333
6                   num__@DurationSet1    0.076293
7                   num__@DurationSet2    0.074555
10  indicator__@DurationSet3_indicator    0.059411
5                            num__Type    0.011572
3              num__@EarningsTotalTeam    0.010180
2               num__@EarnedPointsTeam    0.005999
0                num__@PositionInEntry    0.005763
1                           num__@Rank    0.004309


In [79]:
print(gbr_pipeline.named_steps['gradientboostingregressor'].get_params())

{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 0.8, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


In [80]:
from sklearn.ensemble import RandomForestRegressor
rf_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             RandomForestRegressor(random_state=42, n_estimators=200, max_depth=15, 
                                         min_samples_split=5, min_samples_leaf=2))

RF = rf_pipeline.fit(X_train, y_train)

rf_predictions = rf_pipeline.predict(X_test)

model_scores.append(score_model('RandomForest', y_test, rf_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,3.08,4.02,1.391732e+16,0.678
1,RandomForest,3.27,4.26,1.731130e+16,0.639


In [81]:
print(rf_pipeline.named_steps['randomforestregressor'].get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 15, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [82]:
import joblib
# Modell speichern als .pkl-Datei
joblib.dump(RF, "RandomForest_SpikePoint_OHNEWetter_OhneStatistikDaten.pkl")

['RandomForest_SpikePoint_OHNEWetter_OhneStatistikDaten.pkl']

In [83]:
train_r2 = rf_pipeline.score(X_train, y_train)
test_r2  = rf_pipeline.score(X_test, y_test)
print("Train R²:", train_r2)
print("Test R²:", test_r2)

Train R²: 0.8137874467808649
Test R²: 0.6386751273557476


In [84]:
# 1. Extrahiere das trainierte Model (der letzte Schritt in der Pipeline)
model = rf_pipeline.named_steps['randomforestregressor']

# 2. Extrahiere den Preprocessor-Schritt, der die Features formatiert hat.
# Je nachdem, wie deine Pipeline aufgebaut ist, kann der Schritt-Name variieren.
# Beim make_pipeline werden die Namen automatisch generiert:
preprocessor = rf_pipeline.named_steps[list(rf_pipeline.named_steps.keys())[1]]
# Alternativ, wenn du die Pipeline explizit mit Namen erstellt hast, z.B. im Pipeline-Konstruktur,
# dann wäre es etwas wie:

preprocessor = rf_pipeline.named_steps['columntransformer']

# 3. Hole die Feature-Namen aus dem Preprocessor:
# Diese Methode funktioniert, wenn der Preprocessor und seine Zwischenschritte get_feature_names_out unterstützen.
feature_names = preprocessor.get_feature_names_out()

# 4. Extrahiere die Feature Importances aus dem Model
importances = model.feature_importances_

# 5. Erstelle einen DataFrame zum besseren Überblick
feature_importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importances_df.head(10))


                               feature  importance
9              spec_num__@DurationSet3    0.558039
8                       num__match_win    0.086411
7                   num__@DurationSet2    0.086123
6                   num__@DurationSet1    0.083841
10  indicator__@DurationSet3_indicator    0.027685
3              num__@EarningsTotalTeam    0.020777
0                num__@PositionInEntry    0.020406
2               num__@EarnedPointsTeam    0.018863
1                           num__@Rank    0.018025
5                            num__Type    0.009696


## Cross validation

In [85]:
# param_grids = {
#     "Random Forest Regressor": {
#         "regressor__n_estimators": [50, 100, 200],
#         "regressor__max_depth": [None, 5, 10],
#         "regressor__min_samples_split": [2, 5, 10],
#         "regressor__min_samples_leaf": [1, 2, 4],
#         "regressor__max_features": ["sqrt", "log2"]
#     },
#     "Gradient Boosting Regressor": {
#         "regressor__n_estimators": [50, 100, 200],
#         "regressor__learning_rate": [0.01, 0.1, 0.2],
#         "regressor__max_depth": [3, 5, 7],
#         "regressor__subsample": [0.6, 0.8, 1.0],
#         "regressor__min_samples_split": [2, 5, 10],
#         "regressor__min_samples_leaf": [1, 2, 4]
#     }
# }


In [86]:
# models = {
#     "Random Forest Regressor": Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('regressor', RandomForestRegressor(random_state=42))
#     ]),
#     "Gradient Boosting Regressor": Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('regressor', GradientBoostingRegressor(random_state=42))
#     ])
# }


In [87]:
# ###############################################################################
# # Führe Grid Search CV durch:
# ###############################################################################
# best_models = {}
# best_params_summary = {}

# for model_name, pipeline_model in models.items():
#     print(f"Running GridSearchCV for {model_name}...")
    
#     grid_search = GridSearchCV(
#         estimator=pipeline_model,
#         param_grid=param_grids[model_name],
#         cv=5,
#         scoring='r2',    # Für Regression ist der R²-Score üblich; alternativ könntest du z. B. 'neg_mean_squared_error' verwenden.
#         n_jobs=-1
#     )
    
#     grid_search.fit(X_train, y_train)
    
#     best_models[model_name] = grid_search.best_estimator_
    
#     print(f"Beste Parameter für {model_name}: {grid_search.best_params_}")
#     print(f"Bestes CV R² für {model_name}: {grid_search.best_score_:.4f}\n")

# ###############################################################################
# # Evaluation auf Testdaten (Beispiel für den Random Forest Regressor):
# ###############################################################################
# best_rf = best_models["Random Forest Regressor"]
# y_pred_rf = best_rf.predict(X_test)
# print("Test R² (Random Forest Regressor):", r2_score(y_test, y_pred_rf))

# # Ebenso für Gradient Boosting Regressor:
# best_gb = best_models["Gradient Boosting Regressor"]
# y_pred_gb = best_gb.predict(X_test)
# print("Test R² (Gradient Boosting Regressor):", r2_score(y_test, y_pred_gb))

# ###############################################################################
# # Ausgabe der Zusammenfassung: Beste Parameter (Koordinaten) für alle Modelle
# ###############################################################################
# print("Zusammenfassung der besten Parameter (Koordinaten):")
# for model_name, params in best_params_summary.items():
#     print(f"{model_name}: {params}")